# Web Scraper Tool for 5 US Media Outlets

Import packages

In [2]:
# import packages
import requests
from bs4 import BeautifulSoup

import pandas as pd

### 1. Breitbart - Very Conservative

Get links from the Breitbart Politics homepage

In [56]:
# load the HTML content using requests and save into a variable
r1 = requests.get('https://www.breitbart.com/politics/')
homepage1 = r1.content

# create a soup to allow BeautifulSoup to work
soup1 = BeautifulSoup(homepage1, 'html.parser')

# you can see how complex the homepage is: print(soup1.prettify())
homepage_tags1 = soup1.find_all('h2')

homepage_links = []

for item in homepage_tags1:
    print(item.a['href'])

# remove duplicates by turning list into a set
#homepage_links1 = set(homepage_links1)

/clips/2020/03/04/mccarthy-biden-and-sanders-are-pretty-similar-on-policy/
/clips/2020/03/04/schumer-to-kavanaugh-and-gorsuch-you-will-pay-the-price-wont-know-what-hit-you-if-you-make-awful-decisions/
/politics/2020/03/04/reports-suggest-young-voters-are-failing-to-show-up-for-bernie-sanders-in-droves/
/clips/2020/03/04/dnc-chair-perez-trump-is-the-most-dangerous-president-in-american-history/
/politics/2020/03/04/watch-live-donald-trump-speaks-at-latino-coalition-legislative-summit/
/politics/2020/03/04/pac-backing-elizabeth-warren-wont-place-ad-buys-for-march-10-primaries/
/health/2020/03/04/los-angeles-county-declares-emergency-after-confirming-six-more-coronavirus-cases/
/politics/2020/03/04/establishment-dismisses-bernie-bro-claims-of-a-rigged-election-stop-this-conspiracy-bullsht/
/europe/2020/03/04/the-remainer-deep-state-wants-priti-patels-scalp/
/entertainment/2020/03/04/rob-reiner-joe-biden-will-be-president-when-trump-wakes-from-super-tuesday-nightmare/
/politics/2020/03/04/

TypeError: 'NoneType' object is not subscriptable

### 2. Fox - Conservative

Get links from the Fox Politics homepage

In [3]:
# https://www.foxnews.com/category/politics/2020-presidential-election <- 127 links // 15  without junk
# https://www.foxnews.com/politics <- 150 links (but junk?) // 31 without junk
# i also tried running this for each candidate's section but there is overlap in articles

# load the HTML content using requests and save into a variable
r2 = requests.get('https://www.foxnews.com/politics')
homepage2 = r2.content

# create a soup to allow BeautifulSoup to work
soup2 = BeautifulSoup(homepage2, 'html.parser')

# locate and retrieve all links - WAS NOT ABLE TO ISOLATE JUST ARTICLE LINKS BECUASE NO UNIQUE TAG ELEMENT!!
# you can see how complex the homepage is: print(soup2.prettify())
homepage_tags2 = soup2.find_all('a')
homepage_links2 = []

for link in homepage_tags2:
    homepage_links2.append(link.get('href'))

# remove duplicates by turning list into a set
homepage_links2 = set(homepage_links2)

In [4]:
# remove junk links from list <- MAYBE THIS COULD BE DONE IN A BETTER WAY??
# creating a list of exact junk links that will never be needed when pulling from the homepage
junk_links2 = ['/', '/us', '/world', '/opinion', '/politics', '/entertainment', '//www.foxbusiness.com', 
                  '/lifestyle', '/shows', '//www.foxnews.com/shows/fox-nation', '//radio.foxnews.com/podcast',
                  '#', '//foxnews.com/weather/your-weather/index.html', '#', '#', 
                  '//video.foxnews.com/v/5614615980001/?#sp=watch-live', '#', '/us', '/world', '/opinion', '/politics', 
                  '/official-polls', '/category/politics/elections', 
                  '/entertainment', '//video.foxnews.com/playlist/entertainment-latest-entertainment/', 
                  '//www.foxbusiness.com/', '//www.foxbusiness.com/markets', '//www.foxbusiness.com/politics', 
                  '//www.foxbusiness.com/category/technology', '//www.foxbusiness.com/features', 
                  '//www.foxbusiness.com/category/business-leaders', '/lifestyle', '/food-drink', '/auto', 
                  '/travel', '/family', '/science', '/tech', '/health', '/shows', '/shows', 
                  '/person/personalities', '//video.foxnews.com/v/5614615980001/?#sp=watch-live', 
                  '//video.foxnews.com/playlist/episodic-most-recent-episodes/', 
                  '//video.foxnews.com/#sp=show-clips', '//video.foxnews.com/#sp=news-clips', 
                  '//www.foxnews.com/contact', '//foxcareers.com/Search/SearchResults?brand=Fox%20News%20Careers', 
                  '/foxaroundtheworld/', 'mailto:adsales@foxnews.com?subject=Advertising%20Inquiry', 
                  '//press.foxnews.com/media-contacts/', '//press.foxnews.com/', '/compliance', 
                  'https://supplierdiversity.foxnews.com/', '//www.foxnews.com/shows/fox-nation', '//shop.foxnews.com', 
                  '/go', '//radio.foxnews.com/', '/alerts/subscribe', '/newsletter-signup/alerts', '//radio.foxnews.com/podcast', 
                  '/apps-products', '//www.foxnews.com', '/terms-of-use', '/privacy-policy', '/donotsell', '/closed-captioning', 
                  '//help.foxnews.com', '/contact', '//www.facebook.com/FoxNews', '//twitter.com/foxnews', '//www.google.com/+FoxNews', 
                  '//www.instagram.com/foxnews', '/about/rss/', '/alerts/subscribe', 'https://flipboard.com/@FoxNews', 
                  '//www.foxnews.com/rss/index.html', '//www.foxnews.com/alerts/subscribe.html', 
                   '/accessibility-statement', 'https://video.foxnews.com/v/', 'https://foxnews.com/elections']

# removing all links that lead to section pages AND other specified junk URLs
homepage_links2 = [x for x in homepage_links2 if "/category" not in x] 
homepage_links2 = [x for x in homepage_links2 if "/v/" not in x] 
homepage_links2 = [x for x in homepage_links2 if x not in junk_links2]

In [20]:
# add https://www.foxnews.com to links that don't have this
# create new list for prepared links
hp_links2 = []

for item in homepage_links2:
    new_item = 'https://www.foxnews.com' + item
    hp_links2.append(new_item)

print(hp_links2)

['https://www.foxnews.com/media/chris-matthews-resignation-from-msnbc-hastened-by-series-of-blunders', 'https://www.foxnews.com/politics/warren-huddles-with-advisers-as-progressive-pressure-for-her-to-drop-out-mounts', 'https://www.foxnews.com/politics/supreme-court-at-apparent-odds-over-key-abortion-case-over-clinic-access-restrictions', 'https://www.foxnews.com/politics/fox-news-poll-sanders-knocks-biden-out-of-first-majority-thinks-trump-wins', 'https://www.foxnews.com/politics/issa-mounts-a-comeback-in-california', 'https://www.foxnews.com/politics/biden-campaign-hits-back-at-sanders-ad-showing-obama-praise', 'https://www.foxnews.com/media/francis-tarlov-joe-biden-trump-clinton', 'https://www.foxnews.com/politics/pence-says-passengers-on-flights-from-italy-and-south-korea-will-be-screened-multiple-times-for-coronavirus', 'https://www.foxnews.com/politics/mike-bloomberg-suspends-presidential-campaign-after-super-tuesday-show', 'https://www.foxnews.com/politics/aoc-aligned-progressiv

In [6]:
# TO DO: check for duplicates to set of existing URLs to make sure we only pull data for new ones
    # import full dataset in
    # remove urls in homepage_links3 that are already included in the full dataset

Get article text and other relevent data from new articles

In [7]:
# create a pandas dataframe where all of the retrieved data will be stored
#article_data2 = pd.DataFrame()

#article_data2['headline'] = []
#article_data2['date_published'] = []
#article_data2['date_modified'] = []
#article_data2['description'] = []
#article_data2['author'] = []
#article_data2['url'] = []

#export empty df to file
#article_data2.to_csv('Fox_Data.csv')

In [18]:
text = []
headline = []
date_published = []
date_modified = []
description = []
author = []
url = []

for link in hp_links2:
    # load the HTML content using requests and save into a variable
    r2_article = requests.get(link)
    article2 = r2_article.content

    # create a soups to allow BeautifulSoup to work
    article_soup2 = BeautifulSoup(article2, 'html.parser')

    # get article text
    article_text = article_soup2.find("body").get_text()
    text.append(text)
    
    # retrieve article specific metadata
    metadata2 = article_soup2.find_all("script")[2].get_text()
    metadata2 = metadata2.split(",")

    for item in metadata2:
        if 'headline' in item:
            headline.append(item)
        elif 'datePublished' in item:
            date_published.append(item)
        elif 'dateModified' in item:
            date_modified.append(item)
        elif 'description' in item:
            description.append(item)
        elif 'name' in item and 'Fox News' not in item:
            author.append(item)
        elif 'mainEntityOfPage' in item:
            url.append(item)

In [19]:
print('text', len(text))
print('headline', len(headline))
print('date_published', len(date_published))
print('date_modified', len(date_modified))
print('description', len(description))
print('author', len(author))
print('url', len(url))

text 22
headline 22
date_published 22
date_modified 22
description 22
author 21
url 22


In [9]:
#article_data2 = pd.DataFrame({'text':text,'title':headline, 'date_published':date_published, 'date_modified':date_modified, 'description':description, 'author':author, 'url':url})

ValueError: arrays must all be same length

In [ ]:
# import collection of all responses
all_articles2 = pd.read_csv("Fox_Data.csv", index_col=[0])

# merge new increment of data to the csv
all_articles2 = article_data2.append(article_data2, ignore_index=True)

#export with new increment to save
all_articles.to_csv('Fox_Data.csv')

In [144]:
# another spot the author is located just in case
# test = article_soup3.find_all("meta")[17]
#print(yay)

<meta content="Dana Blanton" data-hid="dc.creator" data-n-head="true" name="dc.creator" scheme="dcterms.creator"/>


### 3. Wall Street Journal - neutral